In [1]:
import pandas as pd
import glob
import datetime
import numpy as np
today_str=str(datetime.datetime.now().date())

In [2]:
folder_file_pattern=glob.glob("/home/jian/Projects/Big_Lots/Sales_All/Sales_Data/*.txt")

In [3]:
taxonomy=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Product Taxonomy.txt",sep="|",dtype=str)
depart_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",sep="|",dtype=str)
class_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Class Names.txt",sep="|",dtype=str,encoding = "ISO-8859-1")

In [4]:
depart_class=taxonomy[['department_id','class_code_id']].drop_duplicates()
depart_class=pd.merge(depart_class,depart_name,on='department_id',how="left")
depart_class=pd.merge(depart_class,class_name,on='class_code_id',how="left")

In [5]:
depart_class.head(2)

,department_id,class_code_id,department_desc,class_code_desc
0,108,11099,CANDYSNACK,FINANCE USE ONLY
1,111,11199,DIRECT STORE DELIVERIES,FINANCE USE ONLY


In [6]:
sales_all=pd.DataFrame()
for file in folder_file_pattern:
    df=pd.read_csv(file,sep="|",dtype=str,na_values="?")
    if (("class_code_id" in df.columns.tolist()) &\
       ("subclass_id" not in df.columns.tolist())):


        df['class_gross_sales_amt']=df['class_gross_sales_amt'].astype(float)
        df['class_gross_sales_amt']=df['class_gross_sales_amt'].fillna(0.0)
        df=df[['location_id','week_end_dt','class_code_id','class_gross_sales_amt']].drop_duplicates()
        df.reset_index(inplace=True)
        try:
            df['week_end_dt']=df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
            
        except:
            print(file,"Date Error")
    elif (("class_code_id" in df.columns.tolist()) &\
       ("subclass_id" in df.columns.tolist())):


        df['subclass_gross_sales_amt']=df['subclass_gross_sales_amt'].astype(float)
        df['subclass_gross_sales_amt']=df['subclass_gross_sales_amt'].fillna(0.0)
        df=df[['location_id','week_end_dt','class_code_id','subclass_id','subclass_gross_sales_amt']].drop_duplicates()
        df=df.groupby(['location_id','week_end_dt','class_code_id'])['subclass_gross_sales_amt'].sum().to_frame().reset_index()
        df=df.rename(columns={"subclass_gross_sales_amt":"class_gross_sales_amt"})
        try:
            df['week_end_dt']=df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
            
        except:
            print(file,"Date Error")
            
            
    else:
        print (file,"Error: class_code_id not in columns")
        
    sales_all=sales_all.append(df)

In [7]:
Q2_sale_2017=sales_all[(sales_all['week_end_dt']>=datetime.datetime(2017,5,13).date()) & (sales_all['week_end_dt']<=datetime.datetime(2017,8,5).date())]
Q2_sale_2018=sales_all[(sales_all['week_end_dt']>=datetime.datetime(2018,5,12).date()) & (sales_all['week_end_dt']<=datetime.datetime(2018,8,6).date())]

In [8]:
Q2_sales_all=Q2_sale_2017.append(Q2_sale_2018)
Q2_sales_all=Q2_sales_all.drop_duplicates()
Q2_sales_all=pd.merge(Q2_sales_all,depart_class,on="class_code_id",how="left")
del Q2_sales_all['index']

In [9]:
Inclusion_stores=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/BL_Sales 2018 Q2 YoY_JL_20180808.xlsx",dtype=str,sheetname="Inclusion Stores",skiprows=1)
Store_Inclusion_List=Inclusion_stores['location_id'].unique().tolist()
Inclusion_stores.head(2)


,location_id,DMA,Q2 Campaign Type,2017-05-13,2017-05-20,2017-05-27,2017-06-03,2017-06-10,2017-06-17,2017-06-24,...,2018-06-02.1,2018-06-09.1,2018-06-16.1,2018-06-23.1,2018-06-30.1,2018-07-07.1,2018-07-14.1,2018-07-21.1,2018-07-28.1,2018-08-04.1
0,1,"COLUMBUS, OH",Q2_DMA,82420.48,74072.4,88270.91,91990.34,77190.27,76737.68,70746.61,...,2712,2416,2449,2372,2444,2819,2874,2339,2151,2248
1,3,AUGUSTA-AIKEN,Balance,65171.78,64123.69,68969.28,78738.19,60562.76,63397.87,59027.22,...,1857,2028,1886,1855,1937,2196,2059,1784,1835,1942


In [10]:
Store_DMA=Inclusion_stores[['location_id','DMA','Q2 Campaign Type']]

In [11]:
Q2_sales_all=pd.merge(Q2_sales_all,Store_DMA,on="location_id",how="left")

In [12]:
Q2_sales_all.head(2)

,class_code_id,class_gross_sales_amt,location_id,week_end_dt,department_id,department_desc,class_code_desc,DMA,Q2 Campaign Type
0,11001,1281.54,1,2017-06-03,108,CANDYSNACK,CANDY,"COLUMBUS, OH",Q2_DMA
1,11003,3215.30,1,2017-06-03,110,BEVGROCERY,BEVERAGES,"COLUMBUS, OH",Q2_DMA


In [13]:
Q2_sales_all_In=Q2_sales_all[Q2_sales_all['location_id'].isin(Store_Inclusion_List)]

In [14]:
test_check=Q2_sales_all_In[Q2_sales_all_In['class_code_id']=="21012"]

In [15]:
test_check.head(2)

,class_code_id,class_gross_sales_amt,location_id,week_end_dt,department_id,department_desc,class_code_desc,DMA,Q2 Campaign Type
65,21012,3922.63,1,2017-06-03,210,LAWN & GARDEN,PATIO FURNITURE,"COLUMBUS, OH",Q2_DMA
281,21012,697.27,1,2017-06-10,210,LAWN & GARDEN,PATIO FURNITURE,"COLUMBUS, OH",Q2_DMA


In [16]:
test_check_2017=test_check[(test_check['week_end_dt']>=datetime.date(2017,1,1))&(test_check['week_end_dt']<=datetime.date(2018,1,1))]
test_check_2017['class_gross_sales_amt'].sum()

57836900.269999996

In [17]:
test_check_2018=test_check[test_check['week_end_dt']>=datetime.date(2018,1,1)]
test_check_2018['class_gross_sales_amt'].sum()

68050997.25999999

In [18]:
test_check['class_gross_sales_amt'].sum()

125887897.53

In [19]:
week_x=pd.read_csv("/home/jian/BigLots/MediaStorm_2018-07-28/MediaStormSalesWeekly20180731-112123-506.txt",dtype=str,sep="|")
week_x['subclass_gross_sales_amt']=week_x['subclass_gross_sales_amt'].astype(float)

week_x[week_x['class_code_id']=="21012"]['subclass_gross_sales_amt'].sum()

2645305.7199999997

In [20]:
Q2_sales_all[(Q2_sales_all['week_end_dt']==datetime.date(2018,7,28)) & (Q2_sales_all['class_code_id']=="21012")]['class_gross_sales_amt'].sum()

2645305.7199999997

In [30]:
Q2_sales_all=Q2_sales_all[['location_id','week_end_dt','DMA','Q2 Campaign Type','department_id','department_desc','class_code_id','class_code_desc','class_gross_sales_amt']]
Q2_sales_all.to_csv("/home/jian/Projects/Big_Lots/Sales_All/BL_Q2 2018 Product Extract_"+today_str+".csv",index=False)



In [31]:
Q2_sales_all_In=Q2_sales_all_In[Q2_sales_all.columns.tolist()]
Q2_sales_all_In.to_csv("/home/jian/Projects/Big_Lots/Sales_All/BL_Q2 2018 Inclusion Stores Product Extract_"+today_str+".csv",index=False)


In [32]:
promoted_list=['21034','60801','21012','21032','61503','36403']
Q2_sales_Promoted=Q2_sales_all_In[Q2_sales_all_In['class_code_id'].isin(promoted_list)]
Q2_sales_Promoted.to_csv("/home/jian/Projects/Big_Lots/Sales_All/BL_Q2 2018 Inclusion Stores Product Promoted_Only Extract_"+today_str+".csv",index=False)

In [39]:
Q2_sales_all[Q2_sales_all['week_end_dt']==datetime.date(2018,7,28)]['class_gross_sales_amt'].sum()

83021811.800000012

In [38]:
week_x['subclass_gross_sales_amt'].sum()

83021811.799999982

In [41]:
week_x[week_x['location_id']!='6990']['subclass_gross_sales_amt'].sum()

82309553.739999995